In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_model_optimization as tfmot


In [10]:
input_shape = [20]
x_train = np.random.randn(1, 20).astype(np.float32)
y_train = tf.keras.utils.to_categorical(np.random.randn(1), num_classes=20)

def setup_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(20, input_shape=input_shape),
        tf.keras.layers.Flatten()
    ])
    return model

def setup_pretrained_weights():
    model = setup_model()
    model.compile(
        loss=tf.keras.losses.categorical_crossentropy,
        optimizer='adam',
        metrics=['accuracy']
    )
    model.fit(x_train, y_train)
    pretrained_weights = './result/model_without_quantization.tf'
    model.save_weights(pretrained_weights)
    return pretrained_weights

def setup_pretrained_model():
    model = setup_model()
    pretrained_weights = setup_pretrained_weights()
    model.load_weights(pretrained_weights)
    return model

setup_model()
pretrained_weights = setup_pretrained_weights()


1/1 [==============================] - 0s 370ms/step - loss: 16.1181 - accuracy: 0.0000e+00


## 量化整个模型

1. 尝试量化某些对模型性能影响不大的神经层
2. 与从头开始训练相比，使用量化感知训练进行微调的效果一般更好 

In [7]:
base_model = setup_model()
base_model.load_weights(pretrained_weights) # optional but recommended for model accuracy

quant_aware_model = tfmot.quantization.keras.quantize_model(base_model)
quant_aware_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer (QuantizeLaye (None, 20)                3         
_________________________________________________________________
quant_dense_10 (QuantizeWrap (None, 20)                425       
_________________________________________________________________
quant_flatten_10 (QuantizeWr (None, 20)                1         
Total params: 429
Trainable params: 420
Non-trainable params: 9
_________________________________________________________________


## 量化某些层

1. 与从头开始训练相比，使用量化感知训练进行微调效果一般更好
2. 尝试量化后面的层而不是前面的层
3. 避免量化关键层，如注意力机制层

In [8]:
# 仅量化Dense层
# Create a base model
base_model = setup_model()
base_model.load_weights(pretrained_weights) # optional but recommended for model accuracy

# Helper function uses `quantize_annotate_layer` to annotate that only the 
# Dense layers should be quantized.
def apply_quantization_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Dense):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

# Use `tf.keras.models.clone_model` to apply `apply_quantization_to_dense` 
# to the layers of the model.
annotated_model = tf.keras.models.clone_model(
    base_model,
    clone_function=apply_quantization_to_dense,
)

# Now that the Dense layers are annotated,
# `quantize_apply` actually makes the model quantization aware.
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
quant_aware_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_1 (QuantizeLa (None, 20)                3         
_________________________________________________________________
quant_dense_11 (QuantizeWrap (None, 20)                425       
_________________________________________________________________
flatten_11 (Flatten)         (None, 20)                0         
Total params: 428
Trainable params: 420
Non-trainable params: 8
_________________________________________________________________


## 创建部署模型

In [11]:
base_model = setup_pretrained_model()
quant_aware_model = tfmot.quantization.keras.quantize_model(base_model)

# Typically you train the model here.

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

1/1 [==============================] - 0s 378ms/step - loss: 16.1181 - accuracy: 0.0000e+00


INFO:tensorflow:Assets written to: C:\Users\wendy\AppData\Local\Temp\tmp67m64bw_\assets


INFO:tensorflow:Assets written to: C:\Users\wendy\AppData\Local\Temp\tmp67m64bw_\assets


### 量化自定义层

In [13]:
LastValueQuantizer = tfmot.quantization.keras.quantizers.LastValueQuantizer
MovingAverageQuantizer = tfmot.quantization.keras.quantizers.MovingAverageQuantizer

class DefaultDenseQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    # Configure how to quantize weights.
    def get_weights_and_quantizers(self, layer):
      return [(layer.kernel, LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]

    # Configure how to quantize activations.
    def get_activations_and_quantizers(self, layer):
      return [(layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
      layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
      layer.activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
      return []

    def get_config(self):
      return {}

In [14]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class CustomLayer(tf.keras.layers.Dense):
  pass

model = quantize_annotate_model(tf.keras.Sequential([
   quantize_annotate_layer(CustomLayer(20, input_shape=(20,)), DefaultDenseQuantizeConfig()),
   tf.keras.layers.Flatten()
]))

# `quantize_apply` requires mentioning `DefaultDenseQuantizeConfig` with `quantize_scope`
# as well as the custom Keras layer.
with quantize_scope(
  {'DefaultDenseQuantizeConfig': DefaultDenseQuantizeConfig,
   'CustomLayer': CustomLayer}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_3 (QuantizeLa (None, 20)                3         
_________________________________________________________________
quant_custom_layer_1 (Quanti (None, 20)                425       
_________________________________________________________________
quant_flatten_16 (QuantizeWr (None, 20)                1         
Total params: 429
Trainable params: 420
Non-trainable params: 9
_________________________________________________________________


### 使用自定义量化算法

In [17]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class FixedRangeQuantizer(tfmot.quantization.keras.quantizers.Quantizer):
  """Quantizer which forces outputs to be between -1 and 1."""

  def build(self, tensor_shape, name, layer):
    # Not needed. No new TensorFlow variables needed.
    return {}

  def __call__(self, inputs, training, weights, **kwargs):
    return tf.keras.backend.clip(inputs, -1.0, 1.0)

  def get_config(self):
    # Not needed. No __init__ parameters to serialize.
    return {}


class ModifiedDenseQuantizeConfig(DefaultDenseQuantizeConfig):
    # Configure weights to quantize with 4-bit instead of 8-bits.
    def get_weights_and_quantizers(self, layer):
      # Use custom algorithm defined in `FixedRangeQuantizer` instead of default Quantizer.
      return [(layer.kernel, FixedRangeQuantizer())]

In [18]:
model = quantize_annotate_model(tf.keras.Sequential([
   # Pass in modified `QuantizeConfig` to modify this `Dense` layer.
   quantize_annotate_layer(tf.keras.layers.Dense(20, input_shape=(20,)), ModifiedDenseQuantizeConfig()),
   tf.keras.layers.Flatten()
]))

# `quantize_apply` requires mentioning `ModifiedDenseQuantizeConfig` with `quantize_scope`:
with quantize_scope(
  {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_4 (QuantizeLa (None, 20)                3         
_________________________________________________________________
quant_dense_16 (QuantizeWrap (None, 20)                423       
_________________________________________________________________
quant_flatten_17 (QuantizeWr (None, 20)                1         
Total params: 427
Trainable params: 420
Non-trainable params: 7
_________________________________________________________________
